In [2]:
import mysql.connector
import pandas as pd
import math
from prettytable import PrettyTable
from sklearn.ensemble import RandomForestRegressor 
password = pd.read_csv("pass.txt")
x = password["password"]
mydb = mysql.connector.connect(host="localhost", user="root", passwd=x[0], database="ecom_database")
mycursor = mydb.cursor()

import getpass
import hashlib
from random import randint
from datetime import datetime, timedelta

def convertpass(password):
    result = hashlib.md5(password.encode())
    return result.hexdigest()

def userlogin():
    username = input("Enter username: ")
    password = getpass.getpass("Enter password: ")
    return username, password

"""CUSTOMER"""##################################################################################################################    

def custregister():
    print("Customer Registration\n")
    print("1. Go to Customer Login Page\n2. Register\n3. Back to homepage\n")
    ch = int(input())
    if (ch == 1):
        custlogin()
    elif (ch == 2):
        CustName = input("Enter name: ")
        CustPhoneNo = input("Enter phone number: ")
        CustAddress = input("Enter address: ")
        CustPincode = input("Enter pincode: ")
        CustEmailID = input("Enter email id: ")
        username = input("Enter username: ")
        password = getpass.getpass("Enter password: ")
        sql = 'SELECT * FROM customer WHERE username = %s'
        val = (username, )
        mycursor.execute(sql, val)
        result = mycursor.fetchall()
        if (len(result) > 0):
            print("username already exists!")
        else:
            mycursor.execute('INSERT INTO customer (CustName, CustPhoneNo, CustAddress, CustPincode, CustEmailID, Username, Password) VALUES (%s, %s, %s, %s, %s, %s, %s)', (CustName, CustPhoneNo, CustAddress, CustPincode, CustEmailID, username, convertpass(password)))
            mydb.commit()
            print("Successfully registered!")
        custregister()
    elif (ch == 3):
        homepage()    
        
def custlogin():
    print("Customer Login\n")
    print("1. Login\n2. Go to Customer Registration Page\n3. Back to homepage\n")
    ch = int(input())
    if (ch == 1):
        username, password = userlogin()
        sql = 'SELECT * FROM customer WHERE username = %s and password = %s'
        val = (username, convertpass(password))
        mycursor.execute(sql, val)
        result = mycursor.fetchall()
        if (len(result) == 0):
            print("No such user exists!")
            custlogin()
        elif (len(result) == 1):
            custpage(result[0])
    elif (ch == 2):
        custregister()
    elif (ch == 3):
        homepage()

def custpage(user):
    
    def vieworders():
        
        def ratedelivery():
            orderID = int(input("Enter OrderID: "))
            DeliveryRating = int(input("Enter Rating (out of 5): "))
            DeliveryReview = input("Write Review (Great/Good/Satisfactory/Average):\n")
            mycursor.execute("select DeliveryManID from orders where OrderID = %s", (orderID, ))
            DeliveryManID = mycursor.fetchone()[0]
            mycursor.execute("insert into delivery_reviews(DeliveryManID, CustID, DeliveryReview, DeliveryRating) values (%s, %s, %s, %s)", (DeliveryManID, user[0], DeliveryReview, DeliveryRating))
            mydb.commit()
            mycursor.execute("select DeliveryManRating from delivery_men where DeliveryManID = %s", (DeliveryManID, ))
            oldDeliveryRating = mycursor.fetchone()[0]
            newDeliveryRating = math.ceil((oldDeliveryRating + DeliveryRating) / 2)
            newDeliverySalary = 5000 * newDeliveryRating
            mycursor.execute("update delivery_men set DeliveryManRating = %s, DeliveryManSalary = %s where DeliveryManID = %s", (newDeliveryRating, newDeliverySalary, DeliveryManID))
            print('Review Submitted')
            vieworders()
        
        def viewspecificproducts():
            
            def rateproduct():
                productID = int(input("Enter ProductID: "))
                ProductRating = int(input("Enter Rating (out of 5): "))
                ProductReview = input("Write Review (Great/Good/Satisfactory/Average):\n")
                mycursor.execute("insert into product_reviews(ProductID, CustID, ProductReview, ProductRating) values (%s, %s, %s, %s)", (productID, user[0], ProductReview, ProductRating))
                mydb.commit()
                mycursor.execute("select ProductRating from product where ProductID = %s", (productID, ))
                oldProductRating = mycursor.fetchone()[0]
                newProductRating = math.ceil((oldProductRating + ProductRating) / 2)
                mycursor.execute("update product set ProductRating = %s where ProductID = %s", (ProductRating, productID))
                print('Review Submitted')
                vieworders()
                
            orderID = int(input("Enter OrderID: "))
            mycursor.execute("select PC.ProductID, P.ProductName, P.ProductPrice, P.ProductRating, PC.ProductQuantity from orders, products_in_cart as PC, product as P where orders.OrderID = %s and orders.CustCartID = PC.CustCartID and PC.ProductID = P.ProductID", (orderID, ))
            specificproducts = mycursor.fetchall();
            y = PrettyTable(['ProductID', 'ProductName', 'ProductPrice', 'ProductRating', 'ProductQuantity'])
            for i in specificproducts:
                y.add_row(i)
            print(y)
            print()
            print("1. Rate/Review Product\n2. Back")
            ch = int(input())
            if (ch == 1):
                rateproduct()
            elif (ch == 2):
                vieworders()
        
        print("Past Orders:-\n")
        x = PrettyTable(['OrderID', 'OrderDate', 'OrderShipDate', 'DeliveryManID', 'TotalAmount', 'TransactionMode'])
        mycursor.execute('select orders.OrderID, orders.OrderDate, orders.OrderShipDate, orders.DeliveryManID, C.TotalAmount, orders.TransactionMode from orders, customer_product_cart as C where C.CustID = %s and orders.OrderShipDate != %s and orders.CustCartID = C.CustCartID', (user[0], 'null'))
        pastorders = mycursor.fetchall()
        for i in pastorders:
            x.add_row(i)
        print(x)
        print()
        print("1. Rate/Review Delivery\n2. View Products in order\n3. Back\n")
        ch = int(input())
        if (ch == 1):
            ratedelivery()
        elif (ch == 2):
            viewspecificproducts()
        elif (ch == 3):    
            custpage(user)
        
    def viewproducts():
        
        def addtocart():
            productid = int(input("Enter product ID: "))
            quantity = int(input("Enter quantity: "))
            sql1 = "select * from product where ProductID = %s"
            val1 = (productid, )
            mycursor.execute(sql1, val1)
            product = mycursor.fetchone()
            mycursor.execute("select CustCartID from customer_product_cart where CustID = %s and CustCartID not in (select CustCartID from orders)", (user[0], ))
            custcart = mycursor.fetchall()
            if (len(custcart) == 0):
                mycursor.execute('insert into customer_product_cart(CustID, TotalAmount) values (%s, %s)', (user[0], product[4]))
                mydb.commit()
                mycursor.execute("select CustCartID from customer_product_cart where CustID = %s and CustCartID not in (select CustCartID from orders)", (user[0], ))
                custcartID = mycursor.fetchone()[0]
                mycursor.execute('insert into products_in_cart values (%s, %s, %s)', (custcartID, productid, quantity))
                mydb.commit()
            else:
                custcartID = custcart[0][0]
                mycursor.execute('insert into products_in_cart values (%s, %s, %s)', (custcartID, productid, quantity))
                mydb.commit()
                mycursor.execute("select TotalAmount from customer_product_cart where CustCartID = %s", (custcartID, ))
                oldTotalAmount = mycursor.fetchone()[0]
                amount = product[4] * quantity
                NewTotalAmount = oldTotalAmount + amount
                mycursor.execute('update customer_product_cart set TotalAmount = %s where CustCartID = %s', (NewTotalAmount, custcartID))
            print("Product successfully added to cart")
            viewproducts() 
            
        def filterproducts():
            print("1. Filter by category\n2. Filter by price\n3. Filter by both category and price\n4. Back")
            fproducts = []
            ch = int(input())
            if (ch == 1):
                mycursor.execute('select ProductCategory from product_categories')
                categories = mycursor.fetchall()
                print("All Categories:-\n")
                for i in categories:
                    print(i[0])
                print()    
                categories = list(input("Choose 1 or more categories: ").split(","))
                for i in categories:
                    mycursor.execute('select ProductCategoryID from product_categories where ProductCategory = %s', (i,))
                    categoryID = mycursor.fetchone()[0]
                    mycursor.execute('select * from product where ProductCategoryID = %s', (categoryID,))
                    fproducts += mycursor.fetchall()
                filteredpdict = {'ID': [], 'Name': [], 'Price': [], 'Seller': [], 'QuantityLeft': [], 'Rating': []}    
                for i in fproducts:
                    filteredpdict['ID'].append(i[0])
                    filteredpdict['Name'].append(i[1])
                    filteredpdict['Price'].append(i[4])
                    filteredpdict['QuantityLeft'].append(i[5])
                    filteredpdict['Rating'].append(i[8])
                    sql2 = "select SellerName from seller where SellerID = %s"
                    val2 = (i[7], )
                    mycursor.execute(sql2, val2)
                    filteredpdict['Seller'].append(mycursor.fetchone()[0])
                df = PrettyTable()    
                for i in filteredpdict.keys():
                    df.add_column(i, filteredpdict[i])
                print (df)
                print()
                filterproducts()
            elif (ch == 2):
                maxprice = int(input("Enter MAX Price: "))
                minprice = int(input("Enter MIN Price: "))
                mycursor.execute('select * from product where ProductPrice between %s and %s', (minprice, maxprice))
                fproducts += mycursor.fetchall()
                filteredpdict = {'ID': [], 'Name': [], 'Price': [], 'Seller': [], 'QuantityLeft': [], 'Rating': []}    
                for i in fproducts:
                    filteredpdict['ID'].append(i[0])
                    filteredpdict['Name'].append(i[1])
                    filteredpdict['Price'].append(i[4])
                    filteredpdict['QuantityLeft'].append(i[5])
                    filteredpdict['Rating'].append(i[8])
                    sql2 = "select SellerName from seller where SellerID = %s"
                    val2 = (i[7], )
                    mycursor.execute(sql2, val2)
                    filteredpdict['Seller'].append(mycursor.fetchone()[0])
                df = PrettyTable()    
                for i in filteredpdict.keys():
                    df.add_column(i, filteredpdict[i])
                print (df)
                print("\n")
                filterproducts()
            elif (ch == 3):
                mycursor.execute('select ProductCategory from product_categories')
                categories = mycursor.fetchall()
                print("All Categories:-\n")
                for i in categories:
                    print(i[0])
                print()    
                categories = list(input("Choose 1 or more categories: ").split(","))
                maxprice = int(input("Enter MAX Price: "))
                minprice = int(input("Enter MIN Price: "))
                for i in categories:
                    mycursor.execute('select ProductCategoryID from product_categories where ProductCategory = %s', (i,))
                    categoryID = mycursor.fetchone()[0]
                    mycursor.execute('select * from product where ProductCategoryID = %s and ProductPrice between %s and %s', (categoryID, minprice, maxprice))
                    fproducts += mycursor.fetchall()
                filteredpdict = {'ID': [], 'Name': [], 'Price': [], 'Seller': [], 'QuantityLeft': [], 'Rating': []}    
                for i in fproducts:
                    filteredpdict['ID'].append(i[0])
                    filteredpdict['Name'].append(i[1])
                    filteredpdict['Price'].append(i[4])
                    filteredpdict['QuantityLeft'].append(i[5])
                    filteredpdict['Rating'].append(i[8])
                    sql2 = "select SellerName from seller where SellerID = %s"
                    val2 = (i[7], )
                    mycursor.execute(sql2, val2)
                    filteredpdict['Seller'].append(mycursor.fetchone()[0])
                df = PrettyTable()
                for i in filteredpdict.keys():
                    df.add_column(i, filteredpdict[i])
                print (df)
                print()
                filterproducts()
            elif (ch == 4):
                viewproducts()
        
        print("All Products:-\n")
        sql1 = "select * from product"
        mycursor.execute(sql1)
        products = mycursor.fetchall()
        pdict = {'ID': [], 'Name': [], 'Price': [], 'Seller': [], 'QuantityLeft': [], 'Rating': []}
        for i in products:
            pdict['ID'].append(i[0])
            pdict['Name'].append(i[1])
            pdict['Price'].append(i[4])
            pdict['QuantityLeft'].append(i[5])
            pdict['Rating'].append(i[8])
            sql2 = "select SellerName from seller where SellerID = %s"
            val2 = (i[7], )
            mycursor.execute(sql2, val2)
            pdict['Seller'].append(mycursor.fetchone()[0])
        df = PrettyTable()
        for i in pdict.keys():
            df.add_column(i, pdict[i])
        print (df)
        print()
        print("1. Add a product to cart\n2. Filter products\n3. Back")
        ch1 = int(input())
        if (ch1 == 1):
            addtocart()
        elif (ch1 == 2):
            filterproducts()
        elif (ch1 == 3):
            custpage(user)
            
    def viewcart():
        
        def placeorder(cartID, cartproducts):
            for i in cartproducts:
                quantity = i[4]
                ProductID = i[0]
                mycursor.execute('select ProductQuantityLeft from product where ProductID = %s', (ProductID, ))
                oldquantity = mycursor.fetchone()[0]
                newquantity = oldquantity - quantity
                mycursor.execute('update product set ProductQuantityLeft = %s where ProductID = %s', (newquantity, ProductID))
            mycursor.execute("Select DeliveryManID from delivery_men")
            ids = mycursor.fetchall()
            num = len(ids) - 1
            index = randint(0, num)
            DeliveryManID = ids[index][0]
            OrderDate = str(datetime.now().month) + '/' + str(datetime.now().day) + '/' + str(datetime.now().year)
            OrderShipDate = 'null'
            ch = int(input("Select Transaction Mode:\n1. NetBanking\n2. Credit Card/Debit Card\n3. Cash On Delivery\n4. Back\n"))
            if (ch == 1):
                TransactionMode = 'NetBanking'
                mycursor.execute('insert into orders(OrderDate, OrderShipDate, CustCartID, DeliveryManID, TransactionMode) values (%s, %s, %s, %s, %s)', (OrderDate, OrderShipDate, cartID, DeliveryManID, TransactionMode))
                mydb.commit()
                print("Order successfully placed!")
                custpage(user)
            elif (ch == 2):    
                TransactionMode = 'Debit Card/Credit Card'
                mycursor.execute('insert into orders(OrderDate, OrderShipDate, CustCartID, DeliveryManID, TransactionMode) values (%s, %s, %s, %s, %s)', (OrderDate, OrderShipDate, cartID, DeliveryManID, TransactionMode))
                mydb.commit()
                print("Order successfully placed!")
                custpage(user)
            elif (ch == 3):
                TransactionMode = 'Cash On Delivery'
                mycursor.execute('insert into orders(OrderDate, OrderShipDate, CustCartID, DeliveryManID, TransactionMode) values (%s, %s, %s, %s, %s)', (OrderDate, OrderShipDate, cartID, DeliveryManID, TransactionMode))
                mydb.commit()
                print("Order successfully placed!")
                custpage(user)
            elif (ch == 4):
                viewcart()
                
        mycursor.execute("select CustCartID from customer_product_cart where CustID = %s and CustCartID not in (select CustCartID from orders)", (user[0], ))
        custcart = mycursor.fetchall()
        if (len(custcart) == 0):
            print("Your cart is empty!\n")
            custpage(user)
        else:
            custcart = custcart[0][0]
            mycursor.execute('select P.ProductID, P.ProductName, P.ProductPrice, P.ProductRating, PC.ProductQuantity from product as P, products_in_cart as PC where PC.CustCartID = %s and P.ProductID = PC.ProductID', (custcart, ))
            TotalPrice = 0
            cartproducts = mycursor.fetchall()
            df = PrettyTable(['ID', 'Name', 'Price', 'Rating', 'Quantity'])
            for i in cartproducts:
                df.add_row(i)
                TotalPrice += i[2] * i[4]
            print (df)
            print()
            print("Total Price:",TotalPrice)
            print()
            print("1. Place Order\n2. Back")
            ch = int(input())
            if (ch == 1):
                placeorder(custcart, cartproducts)
            elif (ch == 2):
                custpage(user)

    
    print("Welcome", user[1]+"\n")
    print("1. View Past Orders\n2. View Products\n3. View Cart\n4. Logout\n")
    ch = int(input())
    if (ch == 1):
        vieworders()
    elif (ch == 2):
        viewproducts()
    elif (ch == 3):
        viewcart()
    elif (ch == 4):
        custlogin()

        

"""SELLER"""####################################################################################################################        
        
def sellregister():
    print("Seller Registration\n")
    print("1. Go to Seller Login Page\n2. Register\n3. Back to homepage\n")
    ch = int(input())
    if (ch == 1):
        selllogin()
    elif (ch == 2):
        SellerName = input("Enter name: ")
        SellerPhoneNo = input("Enter phone number: ")
        SellerAddress = input("Enter address: ")
        SellerPincode = input("Enter pincode: ")
        SellerEmailID = input("Enter email id: ")
        username = input("Enter username: ")
        password = getpass.getpass("Enter password: ")
        sql = 'SELECT * FROM seller WHERE username = %s'
        val = (username, )
        mycursor.execute(sql, val)
        result = mycursor.fetchall()
        if (len(result) > 0):
            print("username already exists!")
        else:
            mycursor.execute('INSERT INTO seller (SellerName, SellerAddress, SellerPhoneNo, SellerPincode, SellerEmailID, Username, Password) VALUES (%s, %s, %s, %s, %s, %s, %s)', (SellerName, SellerAddress, SellerPhoneNo, SellerPincode, SellerEmailID, username, convertpass(password)))
            mydb.commit()
            print("Successfully registered!")
        sellregister()
    elif (ch == 3):
        homepage()    
        
def selllogin():
    print("Seller Login\n")
    print("1. Login\n2. Go to Seller Registration Page\n3. Back to homepage\n")
    ch = int(input())
    if (ch == 1):
        username, password = userlogin()
        sql = 'SELECT * FROM seller WHERE username = %s and password = %s'
        val = (username, convertpass(password))
        mycursor.execute(sql, val)
        result = mycursor.fetchall()
        if (len(result) == 0):
            print("No such user exists!")
            selllogin()
        elif (len(result) == 1):
            sellpage(result[0])
    elif (ch == 2):
        sellregister()
    elif (ch == 3):
        homepage()

def sellpage(user):
    
    def addproduct():
        print(".......ADDING PRODUCT.... PLEASE FILL IN THE PRODUCT DETAILS\n")
        pname = input("ENTER PRODUCT NAME : ")
        print()
        mycursor.execute("select * from product_categories")
        temp1 = PrettyTable(['ID', 'Category'])
        for x in mycursor.fetchall():
            temp1.add_row(x)
        print(temp1)
        print()
        pcid = input("ENTER PRODUCT CATEGORY ID USING ABOVE TABLE (if you cannot find the suitable category for your product, please contact the administrator.) : ")
        pdes = input("ENTER PRODUCT DESCRIPTION : ")
        pprice = input("ENTER PRODUCT PRICE : ")
        pquantity = input("ENTER PRODUCT QUANTITY : ")
        mycursor.execute("select ManufacturerID, CompanyName from manufacturer")
        temp2 = PrettyTable(['ID', 'CompanyName'])
        for x in mycursor.fetchall():
            temp2.add_row(x)
        print(temp2)
        print()
        p_mid = input("ENTER MANUFACTURER'S ID USING ABOVE TABLE: ")
        print()
        sql = 'INSERT INTO product (ProductName, ProductCategoryID, ProductDescription, ProductPrice, ProductQuantityLeft, ManufacturerID, SellerID, ProductRating) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)'
        val = (pname, pcid, pdes, pprice, pquantity, p_mid, user[0], 2)
        mycursor.execute(sql, val)
        mydb.commit()
        print(mycursor.rowcount, "NEW PRODUCT ADDED !!!!!!")
        print("DISPLAYING ALL PRODUCTS LISTED BY YOU ( SELLER ) --> ")
        print()
        sql = "SELECT ProductID, ProductName, ProductPrice, ProductRating FROM product WHERE SellerID = %s"
        mycursor.execute(sql, (user[0],))
        temp3 = PrettyTable(['ProductID', 'ProductName', 'ProductPrice', 'ProductRating'])
        for x in mycursor.fetchall():
            temp3.add_row(x)
        print(temp3)
        print()
        sellpage(user)
        
    def salesoverview():
        print("..........VIEWING SALES OVERVIEW..........")
        print()
        print("Products That You Sell........\n")
        sql = "SELECT ProductID, ProductName, ProductPrice, ProductRating FROM product WHERE SellerID = %s"
        mycursor.execute(sql, (user[0],))
        temp3 = PrettyTable(['ProductID', 'ProductName', 'ProductPrice', 'ProductRating'])
        for x in mycursor.fetchall():
            temp3.add_row(x)
        print(temp3)
        print()
        print(".... SALES OF EACH OF THE PRODUCTS LISTED BY YOU IS BELOW ............\n")
        sql = "select p.ProductID, p.ProductName, p.ProductPrice, p.ProductRating, Sum(pic.ProductQuantity) from products_in_cart as pic, product as p where p.SellerID = %s  AND p.ProductID=pic.ProductID group by p.ProductID"
        mycursor.execute(sql, (user[0],))
        temp3 = PrettyTable(['ProductID', 'ProductName', 'ProductPrice', 'ProductRating', 'QuantitySold'])
        for x in mycursor.fetchall():
            x = list(x)
            x[4] = int(x[4])
            temp3.add_row(x)
        print(temp3)
        print()
        sellpage(user)
        
    def predictprice():
        mycursor.execute("select * from product_categories")
        temp1 = PrettyTable(['ID', 'Category'])
        for x in mycursor.fetchall():
            temp1.add_row(x)
        print(temp1)
        categoryID = input("Which Category ID does Your Product belong to? ")
        mycursor.execute('select ProductID, sum(ProductQuantity) from products_in_cart where CustCartID in (select CustCartID from orders) and ProductID in (select ProductID from product where ProductCategoryID = %s) group by ProductID',(categoryID, ))
        x_train = []
        y_train = []
        products = mycursor.fetchall() 
        for i in products:
            i = list(i)
            i[1] = int(i[1])
            mycursor.execute('select ProductRating, ProductPrice from product where ProductID = %s', (i[0], ))
            i += list(mycursor.fetchone())
            x_train.append(i[1:3])
            y_train.append(i[3])
        minquantity = int(input("Minimum how much quantity of the product to you want to be sold? "))
        rating = int(input("How much rating do you want for the product? "))                  
        x_test = [[minquantity, rating]]
        clf = RandomForestRegressor()
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        y_pred = math.ceil(y_pred[0])
        print("The Predicted Price according to your requirements is", y_pred)
        print()
        sellpage(user)
        
    print("Welcome", user[1]+"\n")
    cart = []
    print("1. Add Product\n2. Sales Overview\n3. Predict Product Prices\n4. Logout\n")
    ch = int(input())
    if (ch == 1):
        addproduct()
    elif (ch == 2):
        salesoverview()
    elif (ch == 3):
        predictprice()
    elif (ch == 4):
        selllogin()

"""MANUFACTURER"""##############################################################################################################        
        
def manuregister():
    print("Manufacturer Registration\n")
    print("1. Go to Manufacturer Login Page\n2. Register\n3. Back to homepage\n")
    ch = int(input())
    if (ch == 1):
        selllogin()
    elif (ch == 2):
        ManuName = input("Enter name: ")
        ManuAddress = input("Enter address: ")
        ManuPincode = input("Enter pincode: ")
        ManuEmailID = input("Enter email id: ")
        username = input("Enter username: ")
        password = getpass.getpass("Enter password: ")
        sql = 'SELECT * FROM manufacturer WHERE username = %s'
        val = (username, )
        mycursor.execute(sql, val)
        result = mycursor.fetchall()
        if (len(result) > 0):
            print("username already exists!")
        else:
            mycursor.execute('INSERT INTO manufacturer (CompanyName, CompanyAddress, CompanyPincode, CompanyEmailID, Username, Password) VALUES (%s, %s, %s, %s, %s, %s)', (ManuName, ManuAddress, ManuPincode, ManuEmailID, username, convertpass(password)))
            mydb.commit()
            print("Successfully registered!")
        manuregister()
    elif (ch == 3):
        homepage()    
        
def manulogin():
    print("Manufacturer Login\n")
    print("1. Login\n2. Go to Manufacturer Registration Page\n3. Back to homepage\n")
    ch = int(input())
    if (ch == 1):
        username, password = userlogin()
        sql = 'SELECT * FROM manufacturer WHERE username = %s and password = %s'
        val = (username, convertpass(password))
        mycursor.execute(sql, val)
        result = mycursor.fetchall()
        if (len(result) == 0):
            print("No such user exists!")
            manulogin()
        elif (len(result) == 1):
            manupage(result[0])
    elif (ch == 2):
        manuregister()
    elif (ch == 3):
        homepage()

def manupage(user):
    
    def viewproducts():
        MF_ID = (user[0], )
        x1 = PrettyTable(["Product ID", "Product Name", "Product Price", "Product Quantity Left", "Product Rating"])
        mycursor.execute("select product.ProductID,  product.ProductName, product.ProductPrice, product.ProductQuantityLeft, product.ProductRating from manufacturer, product where product.ManufacturerID = manufacturer.ManufacturerID and manufacturer.ManufacturerID = %s;", MF_ID)
        productsSold = mycursor.fetchall()
        for i in productsSold:
            x1.add_row(i)
        print(x1)
        print()
        manupage(user)
        
    def viewsellers():
        MF_ID = (user[0], )
        x2 = PrettyTable(["Seller", "PhoneNo", "EmailID", "ProductID", "ProductName", "Price", "Quantity"])
        mycursor.execute("select seller.SellerName, seller.SellerPhoneNo, seller.SellerEmailID, product.ProductID, product.ProductName, product.ProductPrice, product.ProductQuantityLeft from manufacturer, product, seller where product.ManufacturerID = manufacturer.ManufacturerID and manufacturer.ManufacturerID = %s and product.SellerID = seller.SellerID;", MF_ID)
        sellers = mycursor.fetchall()
        for i in sellers:
            x2.add_row(i)
        print(x2)
        print()
        manupage(user)
        
    print("Welcome", user[1]+"\n")
    print("1. View Products\n2. View Sellers\n3. Logout\n")
    ch = int(input())
    if (ch == 1):
        viewproducts()
    elif (ch == 2):
        viewsellers()
    elif(ch == 3):
        manulogin()
        
"""DELIVERYMEN"""###############################################################################################################        
        
def dmanregister():
    print("DeliveryMan Registration\n")
    print("1. Go to DeliveryMan Login Page\n2. Register\n3. Back to homepage\n")
    ch = int(input())
    if (ch == 1):
        dmanlogin()
    elif (ch == 2):
        DManName = input("Enter name: ")
        DManAddress = input("Enter address: ")
        DManPhoneNo = input("Enter phone number: ")
        DManPincode = input("Enter pincode: ")
        DManEmailID = input("Enter email id: ")
        DManSalary = 10000
        DManRating = 2
        username = input("Enter username: ")
        password = getpass.getpass("Enter password: ")
        sql = 'SELECT * FROM delivery_men WHERE username = %s'
        val = (username, )
        mycursor.execute(sql, val)
        result = mycursor.fetchall()
        if (len(result) > 0):
            print("username already exists!")
        else:
            mycursor.execute('INSERT INTO delivery_men (DeliveryManName, DeliveryManAddress, DeliveryManPhoneNo, DeliveryManPincode, DeliveryManEmailID, DeliveryManSalary, DeliveryManRating, Username, Password) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)', (DManName, DManAddress, DManPhoneNo, DManPincode, DManEmailID, DManSalary, DManRating, username, convertpass(password)))
            mydb.commit()
            print("Successfully registered!")
        dmanregister()
    elif (ch == 3):
        homepage()    
        
def dmanlogin():
    print("DeliveryMan Login\n")
    print("1. Login\n2. Go to DeliveryMan Registration Page\n3. Back to homepage\n")
    ch = int(input())
    if (ch == 1):
        username, password = userlogin()
        sql = 'SELECT * FROM delivery_men WHERE username = %s and password = %s'
        val = (username, convertpass(password))
        mycursor.execute(sql, val)
        result = mycursor.fetchall()
        if (len(result) == 0):
            print("No such user exists!")
            dmanlogin()
        elif (len(result) == 1):
            dmanpage(result[0])
    elif (ch == 2):
        dmanregister()
    elif (ch == 3):
        homepage()

def dmanpage(user):
    
    def deliveredorders():
        x3 = PrettyTable(["Order ID", "Customer ID", "Customer Name", "Customer PhoneNo", "Order Date", "Order Ship Date", "Transaction Mode"])
        DM_ID = (user[0], 'null', )
        mycursor.execute("select orders.OrderID, customer.CustID, customer.CustName, customer.CustPhoneNo, orders.OrderDate, orders.OrderShipDate, orders.TransactionMode from orders, customer, customer_product_cart where (orders.DeliveryManID = %s and orders.OrderShipDate != %s and customer_product_cart.CustCartID = orders.CustCartID and customer.CustID = customer_product_cart.CustID)", DM_ID)
        DetailsOfOrdersDelivered = mycursor.fetchall()
        for i in DetailsOfOrdersDelivered:
            x3.add_row(i)
        print(x3)
        print()
    
    def pendingorders():
        x2 = PrettyTable(["OrderID", "CustID", "CustName", "CustPhoneNo", "CustAddress", "OrderDate", "TransactionMode"])
        DM_ID = (user[0], 'null', )
        mycursor.execute("select orders.OrderID, customer.CustID, customer.CustName, customer.CustPhoneNo, customer.CustAddress, orders.OrderDate, orders.TransactionMode from orders, customer, customer_product_cart where (orders.DeliveryManID = %s and orders.OrderShipDate = %s and customer_product_cart.CustCartID = orders.CustCartID and customer.CustID = customer_product_cart.CustID)", DM_ID)
        DetailsOfOrdersNotDelivered = mycursor.fetchall()
        for i in DetailsOfOrdersNotDelivered:
            x2.add_row(i)
        print(x2)
        print()
    
    print("Welcome", user[1]+"\n")
    print("1. View Delivered Orders\n2. View Pending Orders\n3. Logout\n")
    ch = int(input())
    if (ch == 1):
        deliveredorders()
        dmanpage(user)
    elif (ch == 2):
        pendingorders()
        dmanpage(user)
    elif (ch == 3):
        dmanlogin()        

def homepage():
    print("Welcome To Sahi-Rate\n\nChoose one of the following options: \n1. Login as Customer\n2. Login as Seller\n3. Login as Manufacturer\n4. Login as Delivery Worker\n5. Exit")
    ch = int(input())
    if (ch == 1):
        custlogin()
    elif (ch == 2):
        selllogin()
    elif (ch == 3):
        manulogin()
    elif (ch == 4):
        dmanlogin()
    elif (ch == 5):
        return        
        
def userpage():
    
    def adminpage():
        print("1. Display Registered Users\n2. Add a Product Category\n3. Exit\n")
        ch2 = int(input())
        if (ch2 == 1):
            print("Registered Customers\n")
            mycursor.execute('Select CustID, CustName, CustPhoneNo, CustEmailID from customer')
            table = PrettyTable(['ID', 'Name', 'PhoneNo', 'EmailID'])
            for i in mycursor.fetchall():
                table.add_row(i)
            print(table)
            print()
            print("Registered Sellers\n")
            mycursor.execute('Select SellerID, SellerName, SellerPhoneNo, SellerEmailID from seller')
            table = PrettyTable(['ID', 'Name', 'PhoneNo', 'EmailID'])
            for i in mycursor.fetchall():
                table.add_row(i)
            print(table)
            print()
            print("Registered Manufacturers\n")
            mycursor.execute('Select ManufacturerID, CompanyName, CompanyEmailID from manufacturer')
            table = PrettyTable(['ID', 'Name', 'EmailID'])
            for i in mycursor.fetchall():
                table.add_row(i)
            print(table)
            print()
            print("Registered Delivery Workers\n")
            mycursor.execute('Select DeliveryManID, DeliveryManName, DeliveryManPhoneNo, DeliveryManEmailID from delivery_men')
            table = PrettyTable(['ID', 'Name', 'PhoneNo', 'EmailID'])
            for i in mycursor.fetchall():
                table.add_row(i)
            print(table)
            print()
            adminpage()
        elif (ch2 == 2):
            category = input("Enter Product Category: ")
            mycursor.execute('insert into product_categories(ProductCategory) values(%s)', (category, ))
            mydb.commit()
            print("Category successfully added!\n")
            adminpage()
        elif (ch2 == 3):
            return
        
    print("1. Administrator Portal\n2. User Portal\n")
    ch1 = int(input())
    if (ch1 == 1):
        adminpage()
    elif (ch1 == 2):    
        homepage()

userpage()

InterfaceError: 2003: Can't connect to MySQL server on 'localhost:3306' (10061 No connection could be made because the target machine actively refused it)